## Reduction operation: the sum of the numbers in the range [0, value)

In [ ]:
#IMPORTANDO TODAS LAS LIBRERÍAS NECESARIAS
import sys
import numpy as np
import cupy as cp
from numba import cuda

In [ ]:

#captura el parámetro desde la línea de comandos
if len(sys.argv) > 1:
    value=int(sys.argv[1]) #para converir el argumento de la línea de comandos a entero
else:
    value = 5*10**4 #Valor predeterminado si no se pasa ningún parámetro
print(f"El valor de 'value' es: {value}")

In [12]:

print("\n Ejecutando el código original")
def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial
X = np.random.rand(value)

# Para imprimir los pimeros valores del array

# print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Tiempo tardado por reduc operation usando una función:", tiempo)


print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Tiempo tardado por reduc operation usando numpy.sum():", tiempo)

print("Ahora, el resultado utilizando numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Tiempo tardado por reduc operation usando numpy.ndarray.sum():", tiempo)

print("Ahora, el tiempo utilizando numpy.ndarray.sum():", X.sum())


 Ejecutando el código original
Tiempo tardado por reduc operatio usando una función: 48.5 ms ± 343 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
And the result of the sum of numbers in the range [0, value) is: 250342.19152613677

Tiempo tardado por reduc operation usando numpy.sum(): 103 µs ± 69.2 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.sum(): 250342.1915261361 
 
Tiempo tardado por reduc operation usando numpy.ndarray.sum(): 100 µs ± 17.6 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.ndarray.sum(): 250342.1915261361


In [16]:
#Apartado a) Librería Cupy
print("\n Ejecutando el apartado A con CUPY")
# Crear el array en la GPU
X = cp.random.rand(value)

# Suma con CuPy sum()
tiempo = %timeit -r 2 -o -q cp.sum(X)
print("Tiempo tardado por cp.sum():", tiempo)
print(f"El resultado usando cp.sum() es: {cp.sum(X).get()}\n")

# Suma usando el método .sum()
tiempo = %timeit -r 2 -o -q X.sum()
print("Tiempo tardado por X.sum() (CuPy):", tiempo)
print(f"El resultado usando X.sum() (CuPy) es: {X.sum().get()}\n")


Tiempo tardado por cp.sum(): 304 µs ± 2.09 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
El resultado usando cp.sum() es: 4998394.344009519

Tiempo tardado por X.sum() (CuPy): 304 µs ± 3.69 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
El resultado usando X.sum() (CuPy) es: 4998394.344009519



In [15]:
#Apartado B) Librería NUMBA
print("\nEjecutando el apartado B con la librería NUMBA") 
@cuda.jit
def reduc_operation(A, result):
    """Reducción de la suma de los elementos usando CUDA."""
    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    if idx < A.size:
        cuda.atomic.add(result, 0, A[idx])

# Crear el array y enviarlo a la GPU
X = np.random.rand(value).astype(np.float32)
d_X = cuda.to_device(X)
result = cuda.device_array(1, dtype=np.float32)

# Definir los hilos y bloques
threads_per_block = 256
blocks_per_grid = (X.size + threads_per_block - 1) // threads_per_block

# Ejecutar la función de reducción
tiempo = %timeit -r 2 -o -q reduc_operation[blocks_per_grid, threads_per_block](d_X, result)
reduc_operation[blocks_per_grid, threads_per_block](d_X, result)
result_numba = result.copy_to_host()[0]

print("Tiempo tardado por reduc operation usando CUDA:", tiempo)
print(f"El resultado de la suma usando Numba CUDA es: {result_numba}\n")


Ejecutando el apartado B con la librería NUMBA
Tiempo tardado por reduc operation usando CUDA: 16.7 ms ± 19.7 ns per loop (mean ± std. dev. of 2 runs, 1,000 loops each)
El resultado de la suma usando Numba CUDA es: 16777216.0



In [ ]:
#Resultados al mandarlo a la cola BOHR:
#Cuando solo se usa la función reduc_operation el tiempo que tarda en ejecutarse es mucho, esto se debe a que esta poco optimizada.
#Cuando utilizamos la librería CUPY, que es una librería muy parecida a NUmpy pero usando la GPU, para hacerlo más rapido todavía.Esta librería es rapidisima y mejora mucho los tiempos con respecto al código original. Por lo que es un paquete muy recomendable cuando vayamos a trabajar con datos muy grandes y dismpongamos de una GPU
#Por úlimo la librería NUMBA, también trabaja con GPU pero a difrencia de CUPY tambíen lo hace con CPU, por lo que es altamente recomendable para toda clase de trabajos. Numba utiliza CUDA para hacer cálculos en la GPU. Como vemos en la salida del programa el tiempo que se tarda en más o menos igual que con CUPY, pero a mi parecer NUMBA es prefreible porque es más versatil al poder trabajar tanto con CPU como con GPU